<a href="https://colab.research.google.com/github/Sidhtang/30-day-ml-challenge-/blob/main/day_27_working_with_a_few_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 #in most NLP projects, you’ll have access to at least a few labeled examples. The labels
 #might come directly from a client or cross-company team, o

In [2]:
#one simple way to boost the performance of text classifiers on small
#datasets is to apply data augmentation.

In [3]:
# two types of adata augmentation
# back ranslation
 # Take a text in the source language, translate it into one or more target languages
 # using machine translation, and then translate it back to the source language
# token perturbations
 #Given a text from the training set, randomly choose and perform simple trans‐
 #formations like random synonym replacement, word insertion, swap, or
 #deletion.

In [4]:
 #We’ll use the ContextualWordEmbsAug augmenter from NlpAug to leverage the con‐
  #textual word embeddings of DistilBERT for our synonym replacements

In [5]:
!pip install transformers
import transformers

In [6]:
!pip install nlpaug # Install the nlpaug library
from transformers import set_seed
import nlpaug.augmenter.word as naw # Now you can import nlpaug

set_seed(3)
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="substitute")
text = " transfromers are the most popular toys "
print(f" oringinal text: {text}")
print(f" augmented text: {aug.augment(text)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

 oringinal text:  transfromers are the most popular toys 
 augmented text: ['transfromers constitute the most stable components']


In [7]:
 #wraping hte augmentation in a simple function as follows

In [8]:
def augment_text(batch, transformations_per_example=1):
    text_aug, label_ids = [], []
    for text, labels in zip(batch["text"], batch["label_ids"]):
        text_aug += [text]  # Indent this line
        label_ids += [labels] # Indent this line
        for _ in range(transformations_per_example):
            text_aug += [aug.augment(text)] # Indent this line
            label_ids += [labels] # Indent this line
    return {"text": text_aug, "label_ids": label_ids}


In [9]:
# we can use  thus function in our code to train the naive bayes


In [ ]:
# @title  using embeddings as a lookup table
# use the langusge model to embed all labeled texts.#
# perform a nearest neighbor search over the sotred embeddings
#The beauty of this approach is that no model fine-tuning is necessary to leverage the
#few available labeled data points. Instead, the main decision to make this approach
#work is to select an appropriate



In [ ]:
 # lodainng a gpt 2 tokenizet  and  model define the  mean pooling operation


In [11]:
import  torch
from transformers import AutoTokenizer ,AutoModel

In [12]:
model_ckpt = "miguelvictor/python-gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
def mean_pooling(model_output, attention_mask):
 # Extract the token embeddings
 token_embeddings = model_output[0]
 # Compute the attention mask
 input_mask_expanded = (attention_mask
 .unsqueeze(-1)
 .expand(token_embeddings.size())
 .float())
 # Sum the embeddings, but ignore masked tokens
 sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
 sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
 # Return the average as a single vector
 return sum_embeddings / sum_mask


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

In [15]:
def embed_text(examples):
    inputs = tokenizer(examples["text"], padding=True, truncation=True,
                       max_length=128, return_tensors="pt")
    with torch.no_grad():
        # Indent the code block within the 'with' statement
        model_output = model(**inputs)
        pooled_embeds = mean_pooling(model_output, inputs["attention_mask"])
    return {"embedding": pooled_embeds.cpu().numpy()}

In [ ]:
# gpt models dont have inbuil padding token therefore we need to add one \
!pip install datasets
from datasets import load_dataset
ds = load_dataset(dataset_name)


In [16]:
#Now that we have all the embeddings, we need to set up a system to search them. We
 #could write a function that calculates, say, the cosine similarity between a new text
 #embedding that we’ll query and the existing embeddings in the training set. Alterna‐
 # tively, we can use a built-in structure of Datasets called a FAISS index.

  #We already
  #encountered FAISS in Chapter 7. You can think of this as a search engine for embed‐
  #dings,

In [18]:
# this created  a faiss index called embedding

In [20]:
import numpy as np

In [20]:
# we can make use of  functionsget _nearest examples

In [ ]:
def get_sample_preds(sample, m):
 return (np.sum(sample["label_ids"], axis=0) >= m).astype(int)
def find_best_k_m(ds_train, valid_queries, valid_labels, max_k=17):
 max_k = min(len(ds_train), max_k)
 perf_micro = np.zeros((max_k, max_k))
 perf_macro = np.zeros((max_k, max_k))
 for k in range(1, max_k):
 for m in range(1, k + 1):
 _, samples = ds_train.get_nearest_examples_batch("embedding",
 valid_queries, k=k)
 y_pred = np.array([get_sample_preds(s, m) for s in samples])
 clf_report = classification_report(valid_labels, y_pred,
 target_names=mlb.classes_, zero_division=0, output_dict=True)
 perf_micro[k, m] = clf_report["micro avg"]["f1-score"]
 perf_macro[k, m] = clf_report["macro avg"]["f1-score"]
 return perf_micro, perf_macro
